# Wrangling Report

## Table of Contents
- [Introduction](#intro)
- [Gather](#gather)
- [Assess](#assess)
- [Clean](#clean)

<a id='intro'></a>
## Introduction

The main purpose of this report is to walk through the steps taken to gather, assess and clean the proposed dataset. All these three steps are also known as the wrangling phase and the idea is that this process will help make the analysis phase far less troublesome and more straightforward.

<a id='Gather'></a>
## Gather

Data was scattered across three main sources. The way to collect data from those three sources was different in each case and so data collected from each was kept on a separate pandas dataframe. At the end of the gathering process the following dataframes were available

- archive_df: This contains data present in twitter_archive_enhanced.csv. This file was downloaded manually and imported using pandas read_csv method. It features tweets sent by WeRateDogs tweeter account over a period of time. For each tweet it includes information such as numerator rating, denominator rating and type of dog to name a few.

- image_df: This contains predictions of whether main pictures in each tweet corresponds to a dog, and if so which type. The file containing this information was not present in the same folder as the python script and so it was downloaded programatically. This information can be used jointly with archive_df to remove all those entries that are not dogs (according to the neural network used for predictions)

- tweeter_df: We were advised that some of the information contained in archive_df was missing. In this case it was retweet count and favourite count for each of the entries in archive_df. This information was not present in any file, so we had to request these missing pieces from tweeter API.

With all these dataframes ready and loaded we were ready to proceed to the next step

<a id='assess'></a>
## Assess

Alternatively using both visual and programatic inspection I was able to spot a few issues in data available. Some affect data quality and others affect dataframe tidiness. Let's summarize the findings below in bullet points, each point refers to an issue found when assessing the three dataframes available to us.

- A column in *archive_df* indicated whether a particular tweet was a retweet. If a particular tweet is a retweet, then there is a risk the same dog is rated twice (in the original tweet and the retweet). Therefore we need to check whether there instances where for a given retweet the original is present. It was found that there were a few of them in this situation and so they need to be addressed.

- Some of the columns were in the wrong format. Perhaps the most obvious was timestamps that were strings in the orginal dataframe. Also IDs should be integers and not floating point type. This needs addressing too.

- Similarly to the first point, it may well be that tweets that were a reply to another tweet were duplicating information. For example when replying to their own account tweets. This in turn means that again the same dog is rated twice. It was found that there were a few of these and need addressing too.

- The column expanded urls in *archive_df* dataframe contained missing information. Data for this needs to be gathered and so stressing the importance of iterative wrangling.

- It was observed that some of the pet names in the column *names* in *archive_df* were not actually names but rather normal words. It was also seen that the mistakes shared the same trait. They all had the first letter in smaller case. This needs to be taken care of as well in the cleaning phase.

- According to the instructions that we were given, denominators were supposed to be always 10. It was observed that this wasn't the case in the dataframe that was available to us. Therefore something needed to be done.

- Now moving on to image_df if was observed that predictions were not always dogs... Or something dogs were not the first prediction from the neural network. This, as I could observe when looking at the picture for myself was in part due to the fact that a different object was occupying a more relevant position in the picture or was bigger. It makes no sense to have tweets that are not dogs. This needs addressing.

- Again back to *archive_df* the column *source* contains odd-looking content. When digging deeper it can be seen that this column contains values from only four possible categories. However values are wrapped in an xml tag format that needs cleaning.

The 8 issues describe above can be regarded as quality issues. Also two other issues were found that deal more with data structure or tidiness

- There are four columns in *archive_df*, namely *doggo*, *floofer*, *pupper*, *puppo*, that are actually the same variable, however they are distributing occupying four different columns. This is a flaw in the structure of the dataframe and needs addressing.

- Also tweeter_df and archive_df can be regarded as part of the same observational unit. Therefore these two distinct subframes need be joint. This will mean that for the analysis phase there will be only two dataframes to look at.

Once a number of issues have been found, let's proceed to fix them

<a id='clean'></a>
## Clean



#### 1 -> Retweets
The first issue highlighted above was resolved using the code below

In [ ]:
archive_df_clean = archive_df.copy()
archive_df_clean.drop(['retweeted_status_id'],axis=1,inplace=True)
archive_df_clean.rename(columns={'retweeted_status_id_2':'retweeted_status_id'},inplace=True)
retweets = archive_df_clean[archive_df_clean.retweeted_status_id.notnull()]
duplicated = np.isin(retweets['retweeted_status_id'].astype('int64').values,archive_df_clean.tweet_id.values)
retweets = retweets[duplicated]
archive_df_clean.drop(retweets.index,inplace=True)

The idea is to find those retweets whose original is already part of the dataframe. In those cases keep the original and **drop** the retweet. Testing that this issue was corrected can be easily done by checking the number of entries in the dataframe (it should decrease by the same number of duplicate information in retweets)

#### 2 -> Timestamp columns to datetime
Second issue is to cast columns to the appropriate type. For example columns representing timestamps or IDs as ints or string (but not floats)

In [ ]:
archive_df_clean['timestamp'] = pd.to_datetime(archive_df_clean.timestamp)
archive_df_clean['retweeted_status_timestamp'] = pd.to_datetime(archive_df_clean.retweeted_status_timestamp)
archive_df_clean.in_reply_to_user_id = archive_df_clean['in_reply_to_user_id'].astype(str).str.extract('(\d+)')
archive_df_clean.retweeted_status_user_id = archive_df_clean['retweeted_status_user_id'].astype(str).str.extract('(\d+)')

#### 3 -> Replies to tweets
Third issue is very similar to the first one. Need removing all those replies for which an original is already present in the dataframe

In [ ]:
# Get tweet id for which the tweet was a reply
num_rows_before_cleaning_replies = archive_df_clean.shape[0]
archive_df_clean.drop(['in_reply_to_status_id'],axis=1,inplace=True)
archive_df_clean.rename(columns={'in_reply_to_status_id_2':'in_reply_to_status_id'},inplace=True)
replies = archive_df_clean[archive_df_clean.in_reply_to_status_id.notnull()]
duplicated = np.isin(replies['in_reply_to_status_id'].astype('int64').values,archive_df_clean.tweet_id.values)
replies = replies[duplicated]
archive_df_clean.drop(replies.index,inplace=True)

It was tested using the same methodology as for issue \#1

#### 4 -> Missing information in expanded url column
Json data downloaded from tweeter was used to collect all the missing information for this column. The code below resolved the issue

In [ ]:
# Get tweet id for which the tweet was a reply
listTweetsNoUrl = archive_df_clean[archive_df_clean['expanded_urls'].isnull()].tweet_id.values

with open('tweet_json.txt','r',encoding='utf-8') as fjsonRead:
    for line in fjsonRead:
        json_dict = json.loads(line)
        if json_dict['id'] in listTweetsNoUrl:
            archive_df_clean.loc[archive_df_clean[archive_df_clean['tweet_id']==json_dict['id']].index,'expanded_urls'] = json_dict['user']['entities']['url']['urls'][0]['expanded_url']            

#### 5 -> Not all names are pet names
This issue was probably the most tricky. First we listed all those entries that contained a wrong name type. This allowed us to look for patterns that were perhaps not properly used originally. Then and using regular expressions some of the names were correctly extracted and some were not present. For all those that were not present it was decided to use *None*. Then *None* was removed and use np.NaN so it was clear that for some tweets there was no name.

#### 6 -> Denominators different to 10
Again this one was not very straight forward. In general I find that those that deal with regular expressions are probably the most challenging issues to clean. This time, the dataframe was filtered extracting all those entries that did not contain a denominator equals to 10. Then text linked to those entries was analysed visually to check for patterns. It could be seen that some of the wrong entries had two ratios and in fact the second ratio was the correct one. Also, some others it was found that it was not a ratio what was present in the text, but rather a date or 24/7 indicating availability, etc. These were fixed however there were some that we couldn't find a proper reason and so it was decided to scale the numerator to fix the denominator to 10. Code below

In [ ]:
wrongDenSlice = archive_df_clean [ archive_df_clean['rating_denominator'] != 10]
print ('Number of rows with wrong denominator: {}'.format(wrongDenSlice.shape[0]))
wrongDenSlice.loc[:,['text','rating_denominator']]
regexTwoRatings = r'(\d+)\/(\d+).*(\d+)\/(\d+)'
regexDates = r'(\d+)\/(\d+)\/(\d+)'
regex24_7 = r'(24\/7)'
tworatings = wrongDenSlice [ wrongDenSlice.text.str.extract(regexTwoRatings)[3].notnull() ]
num,den = tworatings.text.str.extract(regexTwoRatings)[2],tworatings.text.str.extract(regexTwoRatings)[3]
archive_df_clean.loc[num.index,'rating_numerator'] = num.values
archive_df_clean.loc[den.index,'rating_numerator'] = den.values
# Remove this one as there is no score
dates = wrongDenSlice[wrongDenSlice.text.str.extract(regexDates).notnull()[0].values]
archive_df_clean.drop(dates.index, inplace=True)
data24_7 = wrongDenSlice[ wrongDenSlice.text.str.extract(regex24_7).notnull()[0].values]
archive_df_clean.drop(data24_7.index, inplace=True)
archive_df_clean.drop( archive_df_clean [ archive_df_clean['rating_denominator'] == 0].index, inplace=True)
numeratorToChange = archive_df_clean [ archive_df_clean['rating_denominator'] != 10].index
archive_df_clean.loc[numeratorToChange,'rating_numerator'] = (archive_df_clean.loc[numeratorToChange,'rating_numerator'].astype(int).values/archive_df_clean.loc[numeratorToChange,'rating_denominator'].astype(int).values*10).astype(int)
archive_df_clean.loc[numeratorToChange,'rating_denominator'] = 10

Testing that code work was in this case very easy. Using describe() on denominator column gives clear results

#### 7 -> Predictions are not always dogs
So does it make sense to have tweets that don't contain dogs in the image? Probably not. However some of the tweets did not contain dogs. Also some of the predictions were dogs but were assigned lower probability than the non-dog object. So this needs cleaning as shown below

In [ ]:
def shiftColumns(df,indexToUse,lleft,lright):
    df.loc[indexToUse,lleft[0]] = df.loc[indexToUse,lright[0]]
    df.loc[indexToUse,lleft[1]] = df.loc[indexToUse,lright[1]]
    df.loc[indexToUse,lleft[2]] = df.loc[indexToUse,lright[2]]
    df.loc[indexToUse,lright] = np.NaN
    
image_df_clean = image_df.copy()
# Let's first remove all those rows that do not contain any prediction of a dog
image_df_clean.drop(image_df[ (image_df_clean.p1_dog == False) & (image_df_clean.p2_dog == False) & (image_df_clean.p3_dog == False) ].index, inplace=True)
# Let's now make NaN all those p3_dog == False
image_df_clean.loc[image_df_clean[ (image_df_clean.p3_dog == False) ].index, ['p3','p3_conf','p3_dog']] = np.NaN
# Let's now make NaN all those p2_dog == False
image2Index = image_df_clean[ (image_df_clean.p2_dog == False) ].index
shiftColumns(image_df_clean,image2Index,['p2','p2_conf','p2_dog'],['p3','p3_conf','p3_dog'])
# Let's now make NaN all those p1_dog == False
image1Index = image_df_clean[ (image_df_clean.p1_dog == False) ].index
shiftColumns(image_df_clean,image1Index,['p1','p1_conf','p1_dog'],['p2','p2_conf','p2_dog'])
shiftColumns(image_df_clean,image1Index,['p2','p2_conf','p2_dog'],['p3','p3_conf','p3_dog'])
# Changes booleans to the appropriate type
image_df_clean.p3_dog = image_df_clean.p3_dog.map(lambda x: True if x in [True, 1.0] else np.NaN)
image_df_clean.p2_dog = image_df_clean.p2_dog.map(lambda x: True if x in [True, 1.0] else np.NaN)
image_df_clean.p1_dog = image_df_clean.p1_dog.map(lambda x: True if x in [True, 1.0] else np.NaN)
image_df_clean.loc[1034]

#### 8 -> Properly categorise *source* column
Source column contains categories that are wrapped with xml tags. Remove them to make them clearer

In [ ]:
archive_df_clean.source = archive_df_clean.source.str.extract('(">)(.*)(<\/a)')[1]

Testing it using value counts is sufficient to observe that we got the desired behavior

#### 9 -> Dog type is scattered across four columns
This is the first of the structural problems described above. The idea is to have only one column representing one variable. In this case the variable is dog type. The following code has been used to achieve desired results

In [ ]:
archive_df_clean.replace({'doggo':1,'floofer':1,'pupper':1,'puppo':1},inplace=True)
dogCat = pd.melt(archive_df_clean,id_vars=['tweet_id'],value_vars=['doggo', 'floofer', 'pupper', 'puppo'],var_name='category',value_name='valueToRemove')
dogCat.valueToRemove = dogCat.valueToRemove.replace('None', np.NaN)
dogCat = dogCat.dropna()
archive_df_clean = archive_df_clean.merge(dogCat,how='left',on='tweet_id')
archive_df_clean.drop(['doggo', 'floofer', 'pupper', 'puppo','valueToRemove'],axis=1,inplace=True)

#### 10 -> Two dataframes correspond to the same observational unit
The idea here is to merge two dataframes that contain variables that are actually part of the same observational unit. A join operation, in SQL language, needs to be done for this to happen. Let's look at the code below

In [ ]:
archive_df_clean = archive_df_clean.merge(tweeter_df,on='tweet_id',how='inner')

This is really easy to test by looking at the first few entries of the resulting dataframe and checking that all variables are there.